In [2]:
import os
import sys
import json
import numpy as np
import time
import pandas as pd
from PIL import Image, ImageDraw

In [3]:
scaphoid_dirpath = "Scaphoid"
scaphoid_annotations_dirpath = os.path.join(scaphoid_dirpath, 'Annotations')
scaphoid_images_dirpath = os.path.join(scaphoid_dirpath, 'Images')
scaphoid_slice_data =  os.listdir(os.path.join(scaphoid_annotations_dirpath, 'Scaphoid_Slice'))
fracture_coordinate_data = os.listdir(os.path.join(scaphoid_annotations_dirpath, 'Fracture_Coordinate'))
normal_image_data =  os.listdir(os.path.join(scaphoid_images_dirpath, 'Normal'))
fracture_image_data = os.listdir(os.path.join(scaphoid_images_dirpath, 'Fracture'))
print("Number of normal images :: ", len(normal_image_data))
print("Number of fracture images :: ", len(fracture_image_data))
print("Number of fracture_coordinate :: ", len(fracture_coordinate_data))
print("Number of scaphoid_slice_data :: ", len(scaphoid_slice_data))

Number of normal images ::  120
Number of fracture images ::  120
Number of fracture_coordinate ::  120
Number of scaphoid_slice_data ::  240


In [65]:
target = scaphoid_slice_data[3]
print(target)
target_annotation_path = os.path.join(scaphoid_annotations_dirpath, 'Scaphoid_Slice', target)
if os.path.splitext(target)[0]+'.bmp' in normal_image_data:
    print('normal')
    target_image_path = os.path.join(scaphoid_images_dirpath, 'Normal', os.path.splitext(target)[0]+'.bmp')
else:
    print('fractured')
    target_image_path = os.path.join(scaphoid_images_dirpath, 'Fracture', os.path.splitext(target)[0]+'.bmp')
with open(target_annotation_path) as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()
bboxlist = jsonObject[0]['bbox']
bboxlist.insert(0, target_image_path)
print(bboxlist)

00293717 L 77M AP0.json
normal
['Scaphoid\\Images\\Normal\\00293717 L 77M AP0.bmp', '663', '912', '856', '1091']


In [68]:
datalist = []
for target in scaphoid_slice_data:
    target_annotation_path = os.path.join(scaphoid_annotations_dirpath, 'Scaphoid_Slice', target)
    if os.path.splitext(target)[0]+'.bmp' in normal_image_data:
        #print('normal')
        target_image_path = os.path.join(scaphoid_images_dirpath, 'Normal', os.path.splitext(target)[0]+'.bmp')
    else:
        #print('fractured')
        target_image_path = os.path.join(scaphoid_images_dirpath, 'Fracture', os.path.splitext(target)[0]+'.bmp')
    with open(target_annotation_path) as jsonFile:
        jsonObject = json.load(jsonFile)
        jsonFile.close()
    bboxlist = jsonObject[0]['bbox']
    bboxlist.insert(0, target_image_path)
    datalist.append(bboxlist)
df = pd.DataFrame(datalist, columns =['Filename', 'bbox_topleft_x', 'bbox_topleft_y', 'bbox_botright_x', 'bbox_botright_y'])
df

,Filename,bbox_topleft_x,bbox_topleft_y,bbox_botright_x,bbox_botright_y
0,Scaphoid\Images\Fracture\00075616-AP0.bmp,751,794,873,945
1,Scaphoid\Images\Normal\00169382 L 51F AP0.bmp,679,842,823,984
2,Scaphoid\Images\Normal\00230304 R 50F AP0.bmp,867,943,1007,1092
3,Scaphoid\Images\Normal\00293717 L 77M AP0.bmp,663,912,856,1091
4,Scaphoid\Images\Fracture\00454212-LOB0.bmp,585,996,776,1189
...,...,...,...,...,...
235,Scaphoid\Images\Normal\20101346 L 30F AP0.bmp,709,845,826,1005
236,Scaphoid\Images\Normal\20118837 L 14M AP0.bmp,665,830,831,981
237,Scaphoid\Images\Normal\20123953 R 19Y M AP0.bmp,914,1008,1076,1170
238,Scaphoid\Images\Normal\20126737 L 33 F AP0.bmp,643,858,777,1005


In [70]:
df.to_csv('wholedata.csv',index=False)

In [5]:
normal_image_data.sort()
fracture_image_data.sort()
normal_splits = np.array_split(normal_image_data, 3)
fracture_splits = np.array_split(fracture_image_data, 3)
fold1 = list(normal_splits[0]) + list(fracture_splits[0])
fold2 = list(normal_splits[1]) + list(fracture_splits[1])
fold3 = list(normal_splits[2]) + list(fracture_splits[2])

cv1_train = fold1+fold2
cv1_test = fold3
cv2_train = fold2+fold3
cv2_test = fold1
cv3_train = fold3+fold1
cv3_test = fold2
cv_dataset = {'cv1_train':cv1_train, 'cv1_test':cv1_test, 'cv2_train':cv2_train, 'cv2_test':cv2_test, 'cv3_train':cv3_train, 'cv3_test':cv3_test}

In [96]:
for key, values in cv_dataset.items():
    datalist = []
    for target in values:
        target_name = os.path.splitext(target)[0]
        target_annotation_path = os.path.join(scaphoid_annotations_dirpath, 'Scaphoid_Slice', target_name +'.json')
        if target_name+'.bmp' in normal_image_data:
            #print('normal')
            target_image_path = os.path.join(scaphoid_images_dirpath, 'Normal', target_name+'.bmp')
        else:
            #print('fractured')
            target_image_path = os.path.join(scaphoid_images_dirpath, 'Fracture', target_name+'.bmp')
        with open(target_annotation_path) as jsonFile:
            jsonObject = json.load(jsonFile)
            jsonFile.close()
        bboxlist = jsonObject[0]['bbox']
        bboxlist.insert(0, target_image_path)
        datalist.append(bboxlist)
    df = pd.DataFrame(datalist, columns =['Filename', 'bbox_topleft_x', 'bbox_topleft_y', 'bbox_botright_x', 'bbox_botright_y'])
    df.to_csv(f'{key}.csv',index=False)

In [6]:
for key, values in cv_dataset.items():
    datalist = []
    for target in values:
        target_name = os.path.splitext(target)[0]
        target_annotation_path = os.path.join(scaphoid_annotations_dirpath, 'Scaphoid_Slice', target_name +'.json')
        target_image_path = os.path.join(target_name+'.bmp')
        with open(target_annotation_path) as jsonFile:
            jsonObject = json.load(jsonFile)
            jsonFile.close()
        bboxlist = jsonObject[0]['bbox']
        bboxlist.insert(0, target_image_path)
        datalist.append(bboxlist)
    df = pd.DataFrame(datalist, columns =['Filename', 'bbox_topleft_x', 'bbox_topleft_y', 'bbox_botright_x', 'bbox_botright_y'])
    df.to_csv(f'{key}.csv',index=False)

In [2]:
from PIL import Image, ImageStat

def is_grayscale(path="image.jpg"):
    im = Image.open(path).convert("RGB")
    stat = ImageStat.Stat(im)
    if sum(stat.sum)/3 == stat.sum[0]: #check the avg with any element value
        return True #if grayscale
    else:
        return False #else its colour
is_grayscale('scaphoid_pictures/00075616-AP0.bmp')

True